# Studying the effects of changing the data and running the same models and creating new models

In [ ]:
# Importing the required libraries
import pandas as pd
import numpy as np
import os
os.chdir("..")

from sklearn.cross_validation import train_test_split

In [2]:
enron7 = pd.read_csv("Res/Complete_data_processed/enron7.csv",sep=",")
enron8 = pd.read_csv("Res/Complete_data_processed/enron8.csv",sep=",")
enron9 = pd.read_csv("Res/Complete_data_processed/enron9.csv",sep=",")
enron10 = pd.read_csv("Res/Complete_data_processed/enron10.csv",sep=",")
enron11 = pd.read_csv("Res/Complete_data_processed/enron11.csv",sep=",")
enron12 = pd.read_csv("Res/Complete_data_processed/enron12.csv",sep=",")

In [3]:
# Studying the accuracy using the xgboost models built from enron 1 to enron 6 dataset
new_enron_accuracy_df = pd.DataFrame(columns=["Enron_dataset","model_name","spam","ham","tp","tn","fp","fn"
                                              ,"acc","recall","specificity"])
enron_dataset_list_df = [enron7,enron8,enron9,enron10,enron11,enron12]
new_enron_list = ["Enron"+str(i) for i in range(7,13)]
i=0
j=-1
for enron_dataset in enron_dataset_list_df:
    j+=1
    print("Working on the "+new_enron_list[j])
    model_no_list = range(1,7)
    # Adding some of the attributes that are missing in the data that we created now to test against the models 1 to 6
    for col in ['additiona','aiso','andmanyother','beiieves','couid','dbcaps','enronxgate','hplno',
                        'inciude','inciuding','invoive','lst','maiiings','majordomo','newsietter',
                        'piaced','pians','potentia','projecthoneypot','pubiisher','resuits','shouid',
                        'squirrelmail','technoiogies','technoiogy','utf']:
        enron_dataset[col]=0.0
    for model_no in model_no_list:
        print("Running the model trained from Enron data : "+str(model_no))
        col_attributes = []
        with open("Res/processed_data/attributes_names.txt", 'r') as f:
            col_attributes.extend(f.read().split())
        baseline_X_features = ['line_count', 'token_count', 'punctuations_count', 'single_char_count', 
                                 'number_token_count', 'year_count', 'stopword_count','median_useful_token_len', 
                                 'avg_useful_token_len', 'rareword_count','attributes_len']
        baseline_X_features.extend(col_attributes)

        import pickle
        import xgboost as xgb
        # load model from file
        loaded_xgb = pickle.load(open("Res/"+"enron"+str(model_no)+"_xgboost.pickle.dat", "rb"))
        
        spam=enron_dataset.loc[enron_dataset["mail_label"]=="spam"]
        ham=enron_dataset.loc[enron_dataset["mail_label"]=="ham"]
        
        enron_dataset["label"] = pd.Series(enron_dataset["mail_label"]).apply(lambda x: 1.0 if x=="spam" else 0.0)
        dtest = xgb.DMatrix(enron_dataset[baseline_X_features], label=enron_dataset["label"])
        y_pred = loaded_xgb.predict(dtest)
        y_probas = y_pred # predicted probabilities generated by sklearn classifier
        y_pred = pd.Series(y_pred).apply(lambda x: 1.0 if x>0.5 else 0.0)

        dfs = pd.DataFrame(columns = ["Actual","Pred"])
        dfs["Actual"]=enron_dataset["label"]
        dfs["Pred_probs"]=loaded_xgb.predict(dtest)
        dfs["Pred"] = dfs.apply(lambda row:1.0 if row["Pred_probs"]>0.5 else 0.0,axis=1)

        from sklearn.metrics import confusion_matrix

        y_test = dfs["Actual"]
        y_pred = dfs["Pred"]

        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        print("True Positives(spam) = {}".format(tp))
        print("True Negatives(ham)  = {}".format(tn))
        print("False Positives(actual=ham, predicted=spam) = {}".format(fp))
        print("False Negatives(actual=spam,predicted=ham) = {}".format(fn))
        print("Accuracy = {:.2f}%".format((tp+tn)/float(len(y_test))*100))
        print("Recall = {:.2f}%".format((tp/float(tp+fn))*100))
        print("Specificity = {:.2f}%".format((tn/float(tn+fp))*100))
        
        new_enron_accuracy_df.loc[i]=[new_enron_list[j],"enron"+str(model_no)+"_xgboost",
                                      len(spam),len(ham),tp, tn, fp, fn,
                                      (tp+tn)/float(len(spam)+len(ham))*100,
                                      (tp/float(tp+fn))*100,
                                      (tn/float(tn+fp))*100]
        i+=1
        print("\n")
new_enron_accuracy_df.to_csv("Res/Complete_data_processed/new_enron_accuracy.csv",sep=",",index=False)

Running the model trained from Enron data : 1
True Positives(spam) = 1229
True Negatives(ham)  = 3316
False Positives(actual=ham, predicted=spam) = 346
False Negatives(actual=spam,predicted=ham) = 101
Accuracy = 91.05%
Recall = 92.41%
Specificity = 90.55%


Running the model trained from Enron data : 2
True Positives(spam) = 1068
True Negatives(ham)  = 2994
False Positives(actual=ham, predicted=spam) = 668
False Negatives(actual=spam,predicted=ham) = 262
Accuracy = 81.37%
Recall = 80.30%
Specificity = 81.76%


Running the model trained from Enron data : 3
True Positives(spam) = 877
True Negatives(ham)  = 2604
False Positives(actual=ham, predicted=spam) = 1058
False Negatives(actual=spam,predicted=ham) = 453
Accuracy = 69.73%
Recall = 65.94%
Specificity = 71.11%


Running the model trained from Enron data : 4
True Positives(spam) = 1324
True Negatives(ham)  = 1575
False Positives(actual=ham, predicted=spam) = 2087
False Negatives(actual=spam,predicted=ham) = 6
Accuracy = 58.07%
Recall =

True Positives(spam) = 3497
True Negatives(ham)  = 1409
False Positives(actual=ham, predicted=spam) = 84
False Negatives(actual=spam,predicted=ham) = 695
Accuracy = 86.30%
Recall = 83.42%
Specificity = 94.37%


Running the model trained from Enron data : 4
True Positives(spam) = 4175
True Negatives(ham)  = 790
False Positives(actual=ham, predicted=spam) = 703
False Negatives(actual=spam,predicted=ham) = 17
Accuracy = 87.34%
Recall = 99.59%
Specificity = 52.91%


Running the model trained from Enron data : 5
True Positives(spam) = 4063
True Negatives(ham)  = 936
False Positives(actual=ham, predicted=spam) = 557
False Negatives(actual=spam,predicted=ham) = 129
Accuracy = 87.93%
Recall = 96.92%
Specificity = 62.69%


Running the model trained from Enron data : 6
True Positives(spam) = 3940
True Negatives(ham)  = 1036
False Positives(actual=ham, predicted=spam) = 457
False Negatives(actual=spam,predicted=ham) = 252
Accuracy = 87.53%
Recall = 93.99%
Specificity = 69.39%




In [7]:
# Creating indiviual xgboost models for enron 7 to 12 dataset separately
enron_measure_model_df = pd.DataFrame(columns=["Enron_dataset","spam","ham","train_spam","train_ham",
                                            "test_spam","test_ham","tp","tn","fp","fn","acc","recall","specificity"])

enron_dataset_list_df = [enron7,enron8,enron9,enron10,enron11,enron12]
j=0
for ernron_df in enron_dataset_list_df:
    
    col_attributes=[]
    with open("Res/processed_data/attributes_names.txt", 'r') as f:
            col_attributes.extend(f.read().split())
    # Baseline Features for the logistic regression model
    baseline_X_features = ['line_count', 'token_count', 'punctuations_count', 'single_char_count', 
                         'number_token_count', 'year_count', 'stopword_count','median_useful_token_len', 
                         'avg_useful_token_len', 'rareword_count','attributes_len']
    baseline_X_features.extend(col_attributes)
    Y_feature = "mail_label"

    # Subsetting the data as X and y
    spam = ernron_df[ernron_df["mail_label"]=="spam"]
    ham = ernron_df[ernron_df["mail_label"]=="ham"]

    X_train_spam, X_test_spam, y_train_spam, y_test_spam = \
    train_test_split(spam[baseline_X_features], spam[Y_feature], test_size=0.2, random_state=0)

    X_train_ham, X_test_ham, y_train_ham, y_test_ham = \
    train_test_split(ham[baseline_X_features], ham[Y_feature], test_size=0.2, random_state=0)

    X_train = pd.concat([X_train_spam,X_train_ham])
    X_test = pd.concat([X_test_spam,X_test_ham])
    y_train = pd.concat([y_train_spam,y_train_ham])
    y_test = pd.concat([y_test_spam,y_test_ham])
    y_train["label"] = y_train.apply(lambda x: 1.0 if x=="spam" else 0.0)
    y_test["label"] = y_test.apply(lambda x: 1.0 if x=="spam" else 0.0)

    dtrain = xgb.DMatrix(X_train, label=y_train["label"])
    dtest = xgb.DMatrix(X_test, label=y_test["label"])

    param = {'max_depth': 100, 'eta': 1, 'verbosity': 0, 'objective': 'binary:logistic',"n_estimators": [200]}
    param['nthread'] = 4
    param['eval_metric'] =  ['auc']

    evallist = [(dtest, 'eval'), (dtrain, 'train')]

    num_round = 30
    clf = xgb.train(param, dtrain, num_round, evallist)

    y_pred = clf.predict(dtest)
    y_probas = y_pred # predicted probabilities generated by sklearn classifier
    y_pred = pd.Series(y_pred).apply(lambda x: 1.0 if x>0.5 else 0.0)

    tn, fp, fn, tp = confusion_matrix(y_test["label"], y_pred).ravel()
    print("True Positives(spam) = {}".format(tp))
    print("True Negatives(ham)  = {}".format(tn))
    print("False Positives(actual=ham, predicted=spam) = {}".format(fp))
    print("False Negatives(actual=spam,predicted=ham) = {}".format(fn))
    print("Accuracy = {:.2f}%".format((tp+tn)/float(len(y_test))*100))
    print("Recall = {:.2f}%".format((tp/float(tp+fn))*100))
    print("Specificity = {:.2f}%".format((tn/float(tn+fp))*100))
    
    enron_measure_model_df.loc[j]= ["Enron"+str(j+7),
           len(spam),len(ham),len(X_train_spam),len(X_train_ham),
           len(X_test_spam),len(X_test_ham),
           tp,tn,fp,fn,
           (tp+tn)/float(len(y_test))*100,
           (tp/float(tp+fn))*100,
           (tn/float(tn+fp))*100]
    j+=1
enron_measure_model_df.to_csv("Res/Complete_data_processed/enron_measure_model.csv",sep=",",index=False)

[0]	eval-auc:0.944925	train-auc:0.994251
[1]	eval-auc:0.974228	train-auc:0.998811
[2]	eval-auc:0.981259	train-auc:0.999487
[3]	eval-auc:0.988971	train-auc:0.99988
[4]	eval-auc:0.991066	train-auc:0.999934
[5]	eval-auc:0.991904	train-auc:0.999972
[6]	eval-auc:0.992486	train-auc:0.999986
[7]	eval-auc:0.992707	train-auc:0.99999
[8]	eval-auc:0.992681	train-auc:0.999993
[9]	eval-auc:0.992727	train-auc:0.999993
[10]	eval-auc:0.992758	train-auc:0.999993
[11]	eval-auc:0.993281	train-auc:0.999995
[12]	eval-auc:0.993112	train-auc:0.999996
[13]	eval-auc:0.993235	train-auc:0.999997
[14]	eval-auc:0.993348	train-auc:0.999997
[15]	eval-auc:0.993399	train-auc:0.999997
[16]	eval-auc:0.993527	train-auc:0.999996
[17]	eval-auc:0.993527	train-auc:0.999997
[18]	eval-auc:0.993461	train-auc:0.999997
[19]	eval-auc:0.993343	train-auc:0.999996
[20]	eval-auc:0.993158	train-auc:0.999997
[21]	eval-auc:0.993056	train-auc:0.999997
[22]	eval-auc:0.992733	train-auc:0.999997
[23]	eval-auc:0.992861	train-auc:0.999997
[24]

In [8]:
enron_measure_model_df

,Enron_dataset,spam,ham,train_spam,train_ham,test_spam,test_ham,tp,tn,fp,fn,acc,recall,specificity
0,Enron7,1330,3662,1064,2929,266,733,250,712,21,16,96.200000,93.984962,97.135061
1,Enron8,1290,4356,1032,3484,258,872,250,866,6,8,98.673740,96.899225,99.311927
2,Enron9,1387,2347,1109,1877,278,470,259,456,14,19,95.460614,93.165468,97.021277
3,Enron10,1466,1498,1172,1198,294,300,284,283,17,10,95.294118,96.598639,94.333333
4,Enron11,3878,1499,3102,1199,776,300,767,281,19,9,97.307335,98.840206,93.666667
5,Enron12,4192,1493,3353,1194,839,299,835,285,14,4,98.331870,99.523242,95.317726


In [19]:
new_enron_accuracy_df

,Enron_dataset,model_name,spam,ham,tp,tn,fp,fn,acc,recall,specificity
0,Enron7,enron1_xgboost,1330,3662,1229,3316,346,101,91.045673,92.406015,90.551611
1,Enron7,enron2_xgboost,1330,3662,1068,2994,668,262,81.370192,80.300752,81.758602
2,Enron7,enron3_xgboost,1330,3662,877,2604,1058,453,69.731571,65.939850,71.108684
3,Enron7,enron4_xgboost,1330,3662,1324,1575,2087,6,58.072917,99.548872,43.009285
4,Enron7,enron5_xgboost,1330,3662,1226,2436,1226,104,73.357372,92.180451,66.521027
5,Enron7,enron6_xgboost,1330,3662,1140,2583,1079,190,74.579327,85.714286,70.535227
6,Enron8,enron1_xgboost,1290,4356,1250,3198,1158,40,78.781438,96.899225,73.415978
7,Enron8,enron2_xgboost,1290,4356,1161,4193,163,129,94.828197,90.000000,96.258035
8,Enron8,enron3_xgboost,1290,4356,1238,3372,984,52,81.650726,95.968992,77.410468
9,Enron8,enron4_xgboost,1290,4356,1288,2260,2096,2,62.840949,99.844961,51.882461


In [10]:
print("Completed!")

Completed!
